<a href="https://colab.research.google.com/github/JerryLin0416/PythonTask/blob/main/Taiwan_Mask_Data_from_API_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import requests
import json
import sqlite3
import datetime

In [6]:
url = "https://raw.githubusercontent.com/kiang/pharmacies/master/json/points.json"
response = requests.get(url) # 利用 requests 對 API 來源發送一個請求
d = response.text # 將請求回應的內容存成一個字串格式
data = json.loads(d) # 將長得像 json 格式的字串解析成字典或列表

print(data)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [19]:
med_count = {}

# 填入欄位名稱
for d in data['features']:
    county = d['properties']['address'][0:2] # 取地址前三個字
    if county not in med_count:
      med_count[county] = 1
    else:
      med_count[county] += 1

print(med_count)

{'臺北': 340, '高雄': 424, '臺中': 436, '臺南': 280, '基隆': 57, '新竹': 87, '嘉義': 151, '新北': 507, '桃園': 271, '宜蘭': 76, '苗栗': 57, '彰化': 181, '南投': 67, '雲林': 131, '屏東': 140, '澎湖': 11, '花蓮': 47, '臺東': 23, '金門': 6, '連江': 1}


In [8]:
mask_count = {}

# 填入欄位名稱
for d in data['features']:
    mask = d['properties']['mask_adult']
    county = d['properties']['address'][0:2] # 取地址前三個字
    if county not in mask_count:
      mask_count[county] = mask
    else:
      mask_count[county] += mask

# 將結果從大到小排列
mask_count = dict(sorted(mask_count.items(), key=lambda item: item[1]))

print(mask_count)

{'連江': 7930, '澎湖': 12600, '金門': 15510, '臺東': 28510, '花蓮': 57330, '基隆': 80100, '南投': 91240, '新竹': 99390, '宜蘭': 106900, '苗栗': 134180, '屏東': 184250, '雲林': 199470, '彰化': 219400, '嘉義': 221020, '臺北': 323660, '桃園': 345600, '臺南': 408400, '高雄': 428940, '臺中': 556440, '新北': 677660}


In [26]:
conn = sqlite3.connect('example.db')
c = conn.cursor()

# 新增且清空資料表
c.execute('''CREATE TABLE IF NOT EXISTS pharmacies
             (city text, counts text, createdAt datetime)''')
c.execute('''DELETE FROM pharmacies''')
conn.commit()

# 新增資料
for city, counts in med_count.items():
  t = datetime.datetime.now()
  c.execute(f"INSERT INTO pharmacies VALUES ('{city}', {counts}, '{t}')")
  conn.commit()

# 查詢資料
c.execute("SELECT * FROM pharmacies")
print(c.fetchall())

conn.commit()
conn.close()

[('臺北', '340', '2023-04-15 03:44:01.651496'), ('高雄', '424', '2023-04-15 03:44:01.670256'), ('臺中', '436', '2023-04-15 03:44:01.681302'), ('臺南', '280', '2023-04-15 03:44:01.692262'), ('基隆', '57', '2023-04-15 03:44:01.703289'), ('新竹', '87', '2023-04-15 03:44:01.714277'), ('嘉義', '151', '2023-04-15 03:44:01.723989'), ('新北', '507', '2023-04-15 03:44:01.733934'), ('桃園', '271', '2023-04-15 03:44:01.743245'), ('宜蘭', '76', '2023-04-15 03:44:01.753170'), ('苗栗', '57', '2023-04-15 03:44:01.763840'), ('彰化', '181', '2023-04-15 03:44:01.773892'), ('南投', '67', '2023-04-15 03:44:01.784879'), ('雲林', '131', '2023-04-15 03:44:01.795098'), ('屏東', '140', '2023-04-15 03:44:01.805567'), ('澎湖', '11', '2023-04-15 03:44:01.816087'), ('花蓮', '47', '2023-04-15 03:44:01.826895'), ('臺東', '23', '2023-04-15 03:44:01.838583'), ('金門', '6', '2023-04-15 03:44:01.849310'), ('連江', '1', '2023-04-15 03:44:01.861632')]


In [27]:
conn = sqlite3.connect('example.db')
c = conn.cursor()

# 新增且清空資料表
c.execute('''CREATE TABLE IF NOT EXISTS masks
             (city text, counts text, createdAt datetime)''')
c.execute('''DELETE FROM masks''')
conn.commit()

# 新增資料
for city, counts in mask_count.items():
  t = datetime.datetime.now()
  c.execute(f"INSERT INTO masks VALUES ('{city}', {counts}, '{t}')")
  conn.commit()

# 查詢資料
c.execute("SELECT * FROM masks")
print(c.fetchall())

conn.commit()
conn.close()

[('連江', '7930', '2023-04-15 03:44:03.985222'), ('澎湖', '12600', '2023-04-15 03:44:03.994074'), ('金門', '15510', '2023-04-15 03:44:04.002151'), ('臺東', '28510', '2023-04-15 03:44:04.010414'), ('花蓮', '57330', '2023-04-15 03:44:04.018874'), ('基隆', '80100', '2023-04-15 03:44:04.027086'), ('南投', '91240', '2023-04-15 03:44:04.035981'), ('新竹', '99390', '2023-04-15 03:44:04.043712'), ('宜蘭', '106900', '2023-04-15 03:44:04.053496'), ('苗栗', '134180', '2023-04-15 03:44:04.065850'), ('屏東', '184250', '2023-04-15 03:44:04.074294'), ('雲林', '199470', '2023-04-15 03:44:04.083629'), ('彰化', '219400', '2023-04-15 03:44:04.093223'), ('嘉義', '221020', '2023-04-15 03:44:04.102311'), ('臺北', '323660', '2023-04-15 03:44:04.111119'), ('桃園', '345600', '2023-04-15 03:44:04.119547'), ('臺南', '408400', '2023-04-15 03:44:04.127700'), ('高雄', '428940', '2023-04-15 03:44:04.136088'), ('臺中', '556440', '2023-04-15 03:44:04.148856'), ('新北', '677660', '2023-04-15 03:44:04.164978')]
